# Transform the Ecological Vulnerability indicators

**Ecological exposure** <br>
E1: Percieved change in seagrass quality<br>
E2: Percieved change in seagrass quantity<br>
**Ecological sensitivity**<br>
S1: Local knowledge of association with seagrass<br>
S2: Academic knowledge of assoication with seagrass<br>
**Recovery Potential**<br>
RP1: Percieved quantity change in resource<br>
RP2: Percieved quality change in resource<br>
RP3: Percieved quality change in resource<br>

## Ecological Sensitivity

In [106]:
#import packages
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from zipfile import ZipFile
import os

In [107]:
# load the cleaned data
df = pd.read_csv("/home/rthoms/ocean_dep/sur_A/resource_eco_sens_clean.csv")

In [108]:
df.columns

Index(['Unnamed: 0', '_focal_resource_label', 'sg_dependence',
       'resource_quality_change', 'resource_quantity_change', 'management',
       'compliance', 'ParentGlobalID', 'management_other', 'focal_resource',
       'id', 'date', 'ccp', 'seagrass_quality_change',
       'sg_quality_change_note', 'seagrass_quantity_change',
       'sg_quantity_change_note', 'causes_seagrass_change', 'sg_causes_coded',
       'focal_resources', 'sg_dependence_note', 'resource_quality_change_note',
       'resource_quantity_change_note', 'management_note', 'compliance_note',
       'resource_change_note', 'resource_change_note.1', 'note',
       'review_flag'],
      dtype='object')

In [109]:
# function to pull item from dict
def set_scale(dic,indicator):
    scale=dic[indicator]
    return scale

Indexed value (Xi)= (Actual value - minimum value) / (Maximum value - minimum value)

In [110]:
def index(val, min, max):
    index = (val-min)/(max-min)
    return index

In [111]:
def rescale(val, min0, max0, min1, max1):
    x = (max1-min1)*(val-min0)/(max0-min0)+min1
    return x

## E1
Percieved change in seagrass quality

In [112]:
print(df['seagrass_quality_change'].unique())

['sig_worse' 'no_change']


In [113]:
# create a dictionary to replace descriptive scale values with interger scale values
quality_chage_dic = {'sig_worse':2,'worse':1,'no_change':0, 'better': -1, 'unsure': None}

# apply function
df['E1_scalar']=df.apply(lambda row: set_scale(quality_chage_dic, row['seagrass_quality_change']), axis=1 )
df[['focal_resource','seagrass_quality_change','E1_scalar']]

,focal_resource,seagrass_quality_change,E1_scalar
0,1,sig_worse,2
1,2,sig_worse,2
2,3,sig_worse,2
3,6,sig_worse,2
4,50,sig_worse,2
...,...,...,...
83,50,sig_worse,2
84,55,sig_worse,2
85,56,sig_worse,2
86,64,sig_worse,2


In [114]:
df['E1_index']=df.apply(lambda row: index(row['E1_scalar'],-2,2), axis=1 )
df[['focal_resource','seagrass_quality_change','E1_scalar','E1_index']]


,focal_resource,seagrass_quality_change,E1_scalar,E1_index
0,1,sig_worse,2,1.0
1,2,sig_worse,2,1.0
2,3,sig_worse,2,1.0
3,6,sig_worse,2,1.0
4,50,sig_worse,2,1.0
...,...,...,...,...
83,50,sig_worse,2,1.0
84,55,sig_worse,2,1.0
85,56,sig_worse,2,1.0
86,64,sig_worse,2,1.0


## E2
Percieved change in seagrass quantity


In [115]:
print(df['seagrass_quantity_change'].unique())

['sig_less' 'less']


In [116]:
# create a dictionary to replace descriptive scale values with interger scale values
quanity_chage_dic = {'sig_less':2,'less':1,'no_change':0, 'more': -1, 'sig_more': -2, 'unsure': None}

# apply function
df['E2_scalar']=df.apply(lambda row: set_scale(quanity_chage_dic, row['seagrass_quantity_change']), axis=1 )
df[['focal_resource','seagrass_quantity_change','E2_scalar']]

,focal_resource,seagrass_quantity_change,E2_scalar
0,1,sig_less,2
1,2,sig_less,2
2,3,sig_less,2
3,6,sig_less,2
4,50,sig_less,2
...,...,...,...
83,50,sig_less,2
84,55,sig_less,2
85,56,sig_less,2
86,64,sig_less,2


In [117]:
df['E2_index']=df.apply(lambda row: index(row['E2_scalar'], -2,2), axis=1 )

## S1 
Local knowledge of Association with Seagrass

In [118]:
print(df['sg_dependence'].unique())

['low' 'high' 'none' 'mod']


In [119]:
# create a dictionary to replace descriptive scale values with interger scale values
sg_dep_dic = {'high':3,'mod':2,'low':1,'none': 0}

# apply function
df['S1_scalar']=df.apply(lambda row: set_scale(sg_dep_dic, row['sg_dependence']), axis=1 )
df[['focal_resource','sg_dependence','S1_scalar']]

,focal_resource,sg_dependence,S1_scalar
0,1,low,1
1,2,high,3
2,3,low,1
3,6,low,1
4,50,high,3
...,...,...,...
83,50,mod,2
84,55,high,3
85,56,none,0
86,64,none,0


In [120]:
df['S1_index']=df.apply(lambda row: index(row['S1_scalar'], 0,3), axis=1 )

# S2 
Academic Literature Review of Seagrass Association

In [121]:
# load literature review results
sg_assoc = pd.read_csv("/home/rthoms/ocean_dep/sur_A/seagrass_association_lit_rev.csv")
sg_assoc.columns

Index(['ID', 'Scientific Name',
       'Fishbase Categorization of Seagrass Association',
       'Local Knowledge of Seagrass Association', 'Agreement'],
      dtype='object')

In [122]:
# merge with clean data df
sg_assoc = sg_assoc.rename(columns={'ID': 'focal_resource', 'Fishbase Categorization of Seagrass Association': 'seagrass_assoc_lit'})
df = pd.merge(df,sg_assoc[['focal_resource','seagrass_assoc_lit','Scientific Name']],on='focal_resource', how='left')

In [123]:
# index based on 0-1 scale
df['S2_index']=df.apply(lambda row: index(row['seagrass_assoc_lit'], 0,2), axis=1 )

In [124]:
df

,Unnamed: 0,_focal_resource_label,sg_dependence,resource_quality_change,resource_quantity_change,management,compliance,ParentGlobalID,management_other,focal_resource,...,review_flag,E1_scalar,E1_index,E2_scalar,E2_index,S1_scalar,S1_index,seagrass_assoc_lit,Scientific Name,S2_index
0,0,mihili (1),low,sig_worse,sig_less,other,75_100,b6f6afb4-7c5a-44a1-97d5-74482f766527,veda de area,1,...,0.0,2,1.0,2,1.0,1,0.333333,0.0,Sillago sihama,0.0
1,1,mavionho (2),high,sig_worse,less,other,75_100,b6f6afb4-7c5a-44a1-97d5-74482f766527,Veda de area,2,...,0.0,2,1.0,2,1.0,3,1.000000,1.0,Gerres oyena,0.5
2,2,salani (3),low,sig_worse,sig_less,other,75_100,b6f6afb4-7c5a-44a1-97d5-74482f766527,Veda de area,3,...,0.0,2,1.0,2,1.0,1,0.333333,1.0,Gerres longirostris,0.5
3,3,sigulula (6),low,sig_worse,sig_less,other,75_100,b6f6afb4-7c5a-44a1-97d5-74482f766527,veda de area,6,...,0.0,2,1.0,2,1.0,1,0.333333,2.0,Crenidens crenidens,1.0
4,4,dzindrolo (50),high,sig_worse,sig_less,"gear_closure,other",75_100,b6f6afb4-7c5a-44a1-97d5-74482f766527,NaN,50,...,0.0,2,1.0,2,1.0,3,1.000000,1.0,Portunus pelagicus,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,83,dzindrolo (50),mod,worse,sig_less,gear_closure,100,05d15b5d-f3fa-4464-8670-8bab8e40017c,NaN,50,...,0.0,2,1.0,2,1.0,2,0.666667,1.0,Portunus pelagicus,0.5
84,84,"gudua, girumba (55)",high,no_change,sig_less,gear_closure,100,05d15b5d-f3fa-4464-8670-8bab8e40017c,NaN,55,...,0.0,2,1.0,2,1.0,3,1.000000,2.0,Sepia pharaonis,1.0
85,85,mavandzi (56),none,sig_worse,sig_less,gear_closure,100,05d15b5d-f3fa-4464-8670-8bab8e40017c,NaN,56,...,0.0,2,1.0,2,1.0,0,0.000000,1.0,Penaeidae,0.5
86,86,mahutu (64),none,no_change,no_change,other,100,05d15b5d-f3fa-4464-8670-8bab8e40017c,NaN,64,...,0.0,2,1.0,2,1.0,0,0.000000,0.0,Siliqua polita,0.0


## RP1
Percieved quality change in resource

In [125]:
print(df['resource_quality_change'].unique())

['sig_worse' 'no_change' 'worse' 'better']


In [126]:
# create a dictionary to replace descriptive scale values with interger scale values
quality_chage_dic = {'sig_worse':-2,'worse':-1,'no_change':0, 'better': 1, 'unsure': None}

# apply function
df['RP1_scalar']=df.apply(lambda row: set_scale(quality_chage_dic, row['resource_quality_change']), axis=1 )
df[['focal_resource','resource_quality_change','RP1_scalar']]

,focal_resource,resource_quality_change,RP1_scalar
0,1,sig_worse,-2
1,2,sig_worse,-2
2,3,sig_worse,-2
3,6,sig_worse,-2
4,50,sig_worse,-2
...,...,...,...
83,50,worse,-1
84,55,no_change,0
85,56,sig_worse,-2
86,64,no_change,0


In [127]:
df['RP1_index']=df.apply(lambda row: index(row['RP1_scalar'], -2,2), axis=1 )
df[['focal_resource','resource_quality_change','RP1_scalar','RP1_index']]

,focal_resource,resource_quality_change,RP1_scalar,RP1_index
0,1,sig_worse,-2,0.00
1,2,sig_worse,-2,0.00
2,3,sig_worse,-2,0.00
3,6,sig_worse,-2,0.00
4,50,sig_worse,-2,0.00
...,...,...,...,...
83,50,worse,-1,0.25
84,55,no_change,0,0.50
85,56,sig_worse,-2,0.00
86,64,no_change,0,0.50


## RP2 
Percieved quantity change in fishing resources

In [128]:
print(df['resource_quantity_change'].unique())

['sig_less' 'less' 'no_change' 'unsure' 'sig_more' 'more']


In [129]:
# create a dictionary to replace descriptive scale values with interger scale values
quanity_chage_dic = {'sig_less':-2,'less':-1,'no_change':0, 'more': 1, 'sig_more': 2, 'unsure': None}

# apply scale function
df['RP2_scalar']=df.apply(lambda row: set_scale(quanity_chage_dic, row['resource_quantity_change']), axis=1 )
df[['focal_resource','resource_quantity_change','RP2_scalar']]

,focal_resource,resource_quantity_change,RP2_scalar
0,1,sig_less,-2.0
1,2,less,-1.0
2,3,sig_less,-2.0
3,6,sig_less,-2.0
4,50,sig_less,-2.0
...,...,...,...
83,50,sig_less,-2.0
84,55,sig_less,-2.0
85,56,sig_less,-2.0
86,64,no_change,0.0


In [130]:
df['RP2_index']=df.apply(lambda row: index(row['RP2_scalar'], -2,2), axis=1 )
df[['focal_resource','resource_quantity_change','RP2_scalar','RP2_index']]

,focal_resource,resource_quantity_change,RP2_scalar,RP2_index
0,1,sig_less,-2.0,0.00
1,2,less,-1.0,0.25
2,3,sig_less,-2.0,0.00
3,6,sig_less,-2.0,0.00
4,50,sig_less,-2.0,0.00
...,...,...,...,...
83,50,sig_less,-2.0,0.00
84,55,sig_less,-2.0,0.00
85,56,sig_less,-2.0,0.00
86,64,no_change,0.0,0.50


## RP3 
Managament

In [131]:
print(df['management'].unique())

['other' 'gear_closure,other' 'area_closure' 'none' 'gear_closure'
 'seasonal_harvest' 'species_closure']


In [132]:
# create a dictionary to replace descriptive scale values with interger scale values
management_dic = {'other': 1,'gear_closure':1,'gear_closure,other':1,'area_closure': 1,'gear_closure': 1, 'seasonal_harvest':1, 'species_closure': 1, 'none': 0} 
# apply function
df['RP3_scalar']=df.apply(lambda row: set_scale(management_dic, row['management']), axis=1 )
df[['focal_resource','management','RP3_scalar']]

,focal_resource,management,RP3_scalar
0,1,other,1
1,2,other,1
2,3,other,1
3,6,other,1
4,50,"gear_closure,other",1
...,...,...,...
83,50,gear_closure,1
84,55,gear_closure,1
85,56,gear_closure,1
86,64,other,1


In [133]:
df['RP3_index']=df.apply(lambda row: index(row['RP3_scalar'], 0,1), axis=1 )
df[['focal_resource','management','RP3_scalar','RP3_index']]

,focal_resource,management,RP3_scalar,RP3_index
0,1,other,1,1.0
1,2,other,1,1.0
2,3,other,1,1.0
3,6,other,1,1.0
4,50,"gear_closure,other",1,1.0
...,...,...,...,...
83,50,gear_closure,1,1.0
84,55,gear_closure,1,1.0
85,56,gear_closure,1,1.0
86,64,other,1,1.0


# Combine Indicators

In [134]:
summary = df.groupby(['focal_resource','_focal_resource_label']).agg(
    E1_index=('E1_index', np.mean),
    E2_index=('E2_index', np.mean),
    S1_index=('S1_index', np.mean),
    S2_index=('S2_index', np.mean),
    RP1_index=('RP1_index', np.mean),
    RP2_index=('RP2_index', np.mean),
    RP3_index=('RP3_index', np.mean),
    )

summary['Exposure'] = (summary['E1_index'] + summary['E2_index'])/2
summary['Sensitivity'] = (summary['S1_index'] + summary['S2_index'])/2
summary['Recovery_Potential'] = (summary['RP1_index'] + summary['RP2_index']+ summary['RP3_index'])/3
summary['Ecological_Vulnerability'] = summary['Exposure']  + summary['Sensitivity'] - summary['Recovery_Potential']

summary


,,E1_index,E2_index,S1_index,S2_index,RP1_index,RP2_index,RP3_index,Exposure,Sensitivity,Recovery_Potential,Ecological_Vulnerability
focal_resource,_focal_resource_label,,,,,,,,,,,
1,mihili (1),0.900000,0.975000,0.166667,0.0,0.050000,0.000000,1.000000,0.937500,0.083333,0.350000,0.670833
2,mavionho (2),0.857143,0.964286,0.428571,0.5,0.142857,0.107143,1.000000,0.910714,0.464286,0.416667,0.958333
3,salani (3),0.833333,0.958333,0.277778,0.5,0.083333,0.041667,1.000000,0.895833,0.388889,0.375000,0.909722
4,dzipata (4),1.000000,1.000000,1.000000,1.0,0.500000,0.000000,1.000000,1.000000,1.000000,0.500000,1.500000
6,sigulula (6),0.875000,0.937500,0.750000,1.0,0.250000,0.062500,1.000000,0.906250,0.875000,0.437500,1.343750
7,"carapo, tara (7)",1.000000,1.000000,0.000000,0.0,0.250000,0.000000,1.000000,1.000000,0.000000,0.416667,0.583333
10,tsava (10),1.000000,1.000000,0.000000,0.0,0.500000,0.500000,1.000000,1.000000,0.000000,0.666667,0.333333
11,"sardinha, mboweni (11)",1.000000,1.000000,0.000000,0.0,0.500000,0.500000,0.000000,1.000000,0.000000,0.333333,0.666667
37,nhakusi (37),1.000000,1.000000,0.000000,NaN,0.500000,0.500000,0.000000,1.000000,NaN,0.333333,NaN
